<a href="https://colab.research.google.com/github/Pmilivojevic/Colorization/blob/master/Unet_ResNet_L2AB_256x256.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Activation, Dropout, Add, Concatenate, Input, UpSampling2D, Flatten, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import math
import time

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
'''def generate_random_lines(imshape,slant,drop_length,rain_type):
    drops=[]
    area=imshape[0]*imshape[1]
    no_of_drops=area//1000

    if rain_type.lower()=='drizzle':
        no_of_drops=area//770
        drop_length=10
    elif rain_type.lower()=='heavy':
        drop_length=30
    elif rain_type.lower()=='torrential':
        no_of_drops=area//800
        drop_length=50

    for i in range(no_of_drops): ## If You want heavy rain, try increasing this
        if slant<0:
            x= np.random.randint(slant,imshape[1])
        else:
            x= np.random.randint(0,imshape[1]-slant)
        y= np.random.randint(0,imshape[0]-drop_length)
        drops.append((x,y))
    return drops,drop_length'''

In [0]:
'''def rain_process(image,slant,drop_length,drop_color,drop_width,rain_drops):
    imshape = image.shape  
    image_t= image.copy()
    for rain_drop in rain_drops:
        cv2.line(image_t,(rain_drop[0],rain_drop[1]),(rain_drop[0]+slant,rain_drop[1]+drop_length),drop_color,drop_width)
    image= cv2.blur(image_t,(7,7)) ## rainy view are blurry
    brightness_coefficient = 0.7 ## rainy days are usually shady 
    image_HLS = cv2.cvtColor(image, cv2.COLOR_BGR2HLS) ## Conversion to HLS
    image_HLS[:,:,1] = image_HLS[:,:,1]*brightness_coefficient ## scale pixel values down for channel 1(Lightness)
    image_RGB= cv2.cvtColor(image_HLS, cv2.COLOR_HLS2RGB) ## Conversion to RGB
    return image_RGB'''

In [0]:
'''def add_rain(image,slant=-1,drop_length=20,drop_width=1,drop_color=(200,200,200),rain_type='None'): ## (200,200,200) a shade of gray
    #verify_image(image)
    slant_extreme=slant
    
    imshape = image.shape
    if slant_extreme==-1:
        slant= np.random.randint(-10,10) ##generate random slant if no slant value is given
    rain_drops,drop_length= generate_random_lines(imshape,slant,drop_length,rain_type)
    output= rain_process(image,slant_extreme,drop_length,drop_color,drop_width,rain_drops)
    image_RGB=output    

    return image_RGB'''

In [0]:
'''def add_blur(image, x,y,hw,fog_coeff):
    overlay= image.copy()
    output= image.copy()
    alpha= 0.08*fog_coeff
    rad= hw//2
    point=(x+hw//2, y+hw//2)
    cv2.circle(overlay,point, int(rad), (255,255,255), -1)
    cv2.addWeighted(overlay, alpha, output, 1 -alpha ,0, output)
    return output'''

In [0]:
'''def generate_random_blur_coordinates(imshape,hw):
    blur_points=[]
    midx= imshape[1]//2-2*hw
    midy= imshape[0]//2-hw
    index=1
    while(midx>-hw or midy>-hw):
        for i in range(hw//10*index):
            x= np.random.randint(midx,imshape[1]-midx-hw)
            y= np.random.randint(midy,imshape[0]-midy-hw)
            blur_points.append((x,y))
        midx-=3*hw*imshape[1]//sum(imshape)
        midy-=3*hw*imshape[0]//sum(imshape)
        index+=1
    return blur_points'''

In [0]:
'''def add_fog(image, fog_coeff=-1):
    #verify_image(image)
   # image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    imshape = image.shape
    if fog_coeff==-1:
        fog_coeff_t=random.uniform(0.3,1)
    else:
        fog_coeff_t=fog_coeff
    hw=int(imshape[1]//3*fog_coeff_t)
    haze_list= generate_random_blur_coordinates(imshape,hw)
    for haze_points in haze_list: 
        image= add_blur(image, haze_points[0],haze_points[1], hw,fog_coeff_t) 
    image = cv2.blur(image ,(hw//10,hw//10))
    image_RGB = image

    return image_RGB'''

In [0]:
image_size = 1024
batch_size = 32

In [0]:
data_path= '/content/drive/My Drive/Colab Notebooks/train'

train= []
train_labels= []
j= 0
for i in range(len(os.listdir(data_path))):
  if i < 1500:
#  if i > 1500:
    image= cv2.imread(os.path.join(data_path, os.listdir(data_path)[i]))
    imgLAB= cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    w, h, c= imgLAB.shape
    if w > 255 and h > 255:
      if w >= h:
        img= np.zeros((h,h,3))
        img= imgLAB[np.int((w-h)/2):np.int((w-h)/2+h),:,:]
      else:
        img= np.zeros((w,w,3))
        img= imgLAB[np.int((h-w)/2):np.int((h-w)/2+w),:,:]
      img= cv2.resize(img, (256, 256), cv2.INTER_AREA)
      train.append(img[:,:,0])
      train_labels.append(cv2.merge((img[:,:,1], img[:,:,2])))
        
train= np.array(train).reshape(-1, 256, 256, 1)
train= train/255.0
train_labels= np.array(train_labels).reshape(-1, 256, 256, 2)
train_labels= train_labels/255.0

In [0]:
train.shape

In [0]:
image= cv2.imread(os.path.join(data_path, os.listdir(data_path)[2402]))
plt.imshow(image)
image.shape

In [0]:
#i= np.reshape(train[106], (512, 512))
plt.imshow(imgLAB)

In [0]:
#plt.imshow(train_labels[6])

In [0]:
test_data_path= '/content/drive/My Drive/Colab Notebooks/test'

test= []
test_labels= []

for i in range(len(os.listdir(test_data_path))):
  image= cv2.imread(os.path.join(test_data_path, os.listdir(test_data_path)[i]))
  imgLAB= cv2.cvtColor(cv2.resize(image, (512, 512), cv2.INTER_AREA), cv2.COLOR_BGR2LAB)
  test.append(imgLAB[:,:,0])
  test_labels.append(cv2.merge((imgLAB[:,:,1], imgLAB[:,:,2])))

test= np.array(test).reshape(-1, 512, 512, 1)
test= test/255.0
test_labels= np.array(test_labels).reshape(-1, 512, 512, 2)
test_labels= test_labels/255.0

In [0]:
'''#test_labels[100,:,:,1].shape
L= test[0,:,:,0]*255
A= test_labels[0,:,:,0]*255
B= test_labels[0,:,:,1]*255
img= cv2.merge((L, A, B))
img= np.uint8(img)
#img= cv2.merge((test[0,:,:,0], test_labels[0,:,:,0], test_labels[0,:,:,1]))
img= cv2.cvtColor(img, cv2.COLOR_LAB2RGB)
plt.imshow(img, cmap= 'gray')'''

In [0]:
def bn_act(x, act=True):
    x = BatchNormalization()(x)
    if act == True:
        x = Activation("relu")(x)
    return x

def conv_block(x, filters, kernel_size=(3,3), padding="same", strides=1):
    conv = bn_act(x)
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv

def stem(x, filters, kernel_size=(3,3), padding="same", strides=1):
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    
    shortcut = Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = Add()([conv, shortcut])
    return output

def residual_block(x, filters, kernel_size=(3,3), padding="same", strides=1):
    res = conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    res = conv_block(res, filters, kernel_size=kernel_size, padding=padding, strides=1)
    
    shortcut = Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = Add()([shortcut, res])
    return output

def upsample_concat_block(x, xskip, ks):
    u = UpSampling2D((ks, ks))(x)
    c = Concatenate()([u, xskip])
    return c

In [0]:
def ResUNet():
    f = [16, 32, 64, 256, 512]
    inputs = Input((image_size, image_size, 1))
    
    ## Encoder
    e0 = conv_block(inputs, filters=1, kernel_size=(5,5), padding='same', strides=4)
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=4)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=4)
    e5 = residual_block(e4, f[4], strides=2)
    
    ## Bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)
    
    ## Decoder
    u1 = upsample_concat_block(b1, e4, 2)
    d1 = residual_block(u1, f[4])
    
    u2 = upsample_concat_block(d1, e3, 4)
    d2 = residual_block(u2, f[3])
    
    u3 = upsample_concat_block(d2, e2, 2)
    d3 = residual_block(u3, f[2])
    
    u4 = upsample_concat_block(d3, e1, 4)
    d4 = residual_block(u4, f[1])
    
    outputs = Conv2D(2, (1, 1), padding="same", activation="sigmoid")(d4)
    outputs= UpSampling2D((4, 4))(outputs)
    model = Model(inputs, outputs)
    return model

In [0]:
def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true)))
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

In [0]:
model = ResUNet()
adam = Adam()
model.compile(Adam(lr= .0001), loss= 'mean_squared_error', metrics=[r2_score])
model.summary()

In [0]:
models_path= '/content/drive/My Drive/Colab Notebooks/models/256x256'
len(os.listdir(models_path))
model_load_path= models_path + '/model_' + str(len(os.listdir(models_path))) + '.h5'
#model_load_path= models_path + '/model_11.h5'
#print(model_load_path)

In [0]:
dependencies= {
    'r2_score': r2_score
}
model= load_model(model_load_path, custom_objects= dependencies)
#model.summary()

In [0]:
model.summary()

In [0]:
model.optimizer= Adam(learning_rate= 0.00001)

In [0]:
model_save_path= models_path + '/model_' + str(len(os.listdir(models_path))+1) + '.h5'
model.fit(train, train_labels, batch_size= batch_size, validation_split= 0.1, epochs=50)
model.save(model_save_path)

In [0]:
img= cv2.cvtColor(cv2.merge((np.uint8(test[60,:,:,0]*255), np.uint8(test_labels[60,:,:,0]*255), np.uint8(test_labels[60,:,:,1]*255))), cv2.COLOR_LAB2RGB)
plt.imshow(img, cmap= 'gray')

In [0]:
predictions= model.predict(test)

In [0]:
img= cv2.cvtColor(cv2.merge((np.uint8(test[67,:,:,0]*255), np.uint8(predictions[67,:,:,0]*255), np.uint8(predictions[67,:,:,1]*255))), cv2.COLOR_LAB2RGB)
plt.imshow(img)

In [0]:
plt.imshow(test[60,:,:,0], cmap= 'gray')

In [0]:
test_image= cv2.imread('/content/drive/My Drive/Colab Notebooks/Sofija.bmp')
#test_image= test_image[70:442, 70:442]
test_image= cv2.resize(test_image, (256, 256), cv2.INTER_AREA)
test_image= cv2.cvtColor(test_image, cv2.COLOR_RGB2GRAY)

In [0]:
plt.imshow(test_image, cmap='gray')

In [0]:
test_image= np.array(test_image).reshape(-1, 256, 256, 1)
test_image= test_image/255.0

In [0]:
t= time.time()
prediction= model.predict(test_image)
print(time.time()-t)

In [0]:
img= cv2.cvtColor(cv2.merge((np.uint8(test_image[0,:,:,0]*255), np.uint8(prediction[0,:,:,0]*255), np.uint8(prediction[0,:,:,1]*255))), cv2.COLOR_LAB2RGB)
plt.imshow(img)

In [0]:
B_model_load_path= models_path + '/model_11.h5'
B_model= load_model(B_model_load_path, custom_objects= dependencies)
B_prediction= B_model.predict(test_image)
img= cv2.cvtColor(cv2.merge((np.uint8(test_image[0,:,:,0]*255), np.uint8(B_prediction[0,:,:,0]*255), np.uint8(B_prediction[0,:,:,1]*255))), cv2.COLOR_LAB2RGB)
plt.imshow(img, cmap= 'gray')